In [15]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
import plotly.graph_objects as go
from sklearn.preprocessing import MultiLabelBinarizer
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')
from clustering_pre_post_toxicdetection import load_toxic_combinations_from_single_file, get_privilege_level, binarize_entitlement_data_by_department, clean_binarized_data_with_toxic_combinations,perform_clustering_analysis, create_detailed_comparison_summary, complete_entitlement_cleaning_pipeline

In [16]:
data_toxic=pd.read_csv('data/data.csv')

In [19]:
data_toxic.head()

,Unnamed: 0,employee_id,entitlement_id,grant_date,entitlement_name,application_id,type,entitlement_description,risk_level,function,department_id,manager_id,name_department,parent_id,name,app_description,criticality
0,0,E001,ENT1292,2024-02-16,Admin_TradingPlatformAsianotifications,A033,admin,MODIFY_accounts_TRADINGPLATFORM ASIA,8,TradingPlatformAsianotifications,D011,NaN,Treasury,D002,TradingPlatform Asia,Financial instrument trading system - Asia reg...,9
1,1,E001,ENT171,2024-07-26,Admin_FXpositions,A005,admin,Define FX positions in TreasuryManagement,8,FXpositions,D011,NaN,Treasury,D002,TreasuryManagement,Cash and liquidity management platform,7
2,2,E001,ENT486,2021-10-30,Read_MobileBankingnotifications,A013,read,Retrieve MobileBanking notifications in Mobile...,4,MobileBankingnotifications,D011,NaN,Treasury,D002,MobileBanking,Mobile banking application platform,7
3,3,E001,ENT1368,2025-02-08,Admin_TradingPlatformAmericasconfigurationsett...,A035,admin,Control TradingPlatform Americas configuration...,8,TradingPlatformAmericasconfigurationsettings,D011,NaN,Treasury,D002,TradingPlatform Americas,Financial instrument trading system - Americas...,9
4,4,E001,ENT1774,2023-04-26,Write_CardManagementAsiaconfigurationsettings,A045,write,User access,6,CardManagementAsiaconfigurationsettings,D011,NaN,Treasury,D002,CardManagement Asia,Credit and debit card issuance and management ...,8


In [20]:
data_toxic=data_toxic.drop('Unnamed: 0', axis=1)

In [23]:
entitlements_sets = data_toxic.groupby(['employee_id'])['entitlement_id'].apply(set)

# Reset index to get both employee_id and employee_name as columns
entitlements_sets = entitlements_sets.reset_index()
#print(entitlements_sets)

# Initialize and fit the binarizer on the 'entitlements' column
mlb = MultiLabelBinarizer()
entitlements_binarized = mlb.fit_transform(entitlements_sets['entitlement_id'])

# Create binarized DataFrame
binarized_df = pd.DataFrame(
    entitlements_binarized,
    columns=mlb.classes_,
)

# Insert Employee_ID at the front
binarized_df.insert(0, 'employee_id', entitlements_sets['employee_id'].values)

In [24]:
binarized_df.head()

,employee_id,ENT001,ENT002,ENT003,ENT004,ENT005,ENT006,ENT007,ENT008,ENT009,...,ENT990,ENT991,ENT992,ENT993,ENT994,ENT995,ENT996,ENT997,ENT998,ENT999
0,E001,0,1,0,1,1,1,1,1,1,...,1,0,0,0,0,0,1,0,1,0
1,E002,0,1,0,1,1,1,1,0,0,...,0,0,1,0,0,0,0,0,1,0
2,E003,0,1,0,0,1,1,0,0,0,...,1,0,1,0,0,0,1,0,1,1
3,E004,1,1,0,1,1,1,1,1,1,...,0,0,1,0,0,0,1,0,1,0
4,E005,1,0,0,0,1,0,1,0,1,...,0,1,0,0,1,1,1,0,0,0


In [25]:
def prepare_full_employee_data(data):
    """
    Prepare the full employee dataset for clustering (not by department).

    Args:
        data: DataFrame with employee_id, ENT001-ENT999, metadata columns

    Returns:
        X: Feature matrix (entitlement columns only)
        feature_cols: List of entitlement column names
        metadata: DataFrame with employee metadata
    """
    print("📊 Preparing full employee dataset for clustering...")

    # Identify entitlement columns efficiently
    entitlement_cols = [col for col in data.columns if col.startswith('ENT') and len(col) == 6 and col[3:].isdigit()]
    entitlement_cols.sort()

    # Metadata columns to preserve
    metadata_cols = ['employee_id', 'department_id', 'manager_id', 'name_department',
                    'function', 'avg_risk', 'max_risk', 'total_entitlements',
                    'avg_criticality', 'max_criticality']
    metadata_cols = [col for col in metadata_cols if col in data.columns]

    print(f"✅ Found {len(entitlement_cols)} entitlement columns")
    print(f"✅ Processing {len(data)} employees")

    # Efficient extraction using numpy
    X = data[entitlement_cols].values.astype(np.float32)  # Use float32 for memory efficiency
    metadata = data[metadata_cols]

    return X, entitlement_cols, metadata

def find_optimal_k_efficient(X, k_range=None, max_k=15):
    """
    Efficiently find optimal k using silhouette score with optimized computation.

    Args:
        X: Feature matrix
        k_range: List of k values to test
        max_k: Maximum k to test

    Returns:
        Dict with results and best k
    """
    print("🔍 Finding optimal k using efficient silhouette analysis...")

    # Scale features once
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X).astype(np.float32)

    # Determine k range
    if k_range is None:
        max_k = min(max_k, int(np.sqrt(len(X))))  # Rule of thumb: sqrt(n)
        k_range = range(2, max_k + 1)

    results = {
        'k_values': [],
        'silhouette_scores': [],
        'cluster_labels': {},
        'best_k': None,
        'best_silhouette': -1
    }

    for k in k_range:
        print(f"  Testing k={k}...", end='')

        # K-means with optimized parameters
        kmeans = KMeans(
            n_clusters=k,
            random_state=42,
            n_init=5,  # Reduced for speed
            max_iter=100,  # Usually converges faster
            algorithm='elkan'  # Faster for well-separated clusters
        )

        labels = kmeans.fit_predict(X_scaled)

        # Calculate silhouette score
        if len(np.unique(labels)) > 1:
            # Sample for large datasets to speed up silhouette calculation
            if len(X) > 5000:
                sample_size = min(5000, len(X))
                sample_indices = np.random.choice(len(X), sample_size, replace=False)
                sil_score = silhouette_score(X_scaled[sample_indices], labels[sample_indices])
            else:
                sil_score = silhouette_score(X_scaled, labels)
        else:
            sil_score = -1

        results['k_values'].append(k)
        results['silhouette_scores'].append(sil_score)
        results['cluster_labels'][k] = labels

        print(f" Silhouette: {sil_score:.4f}")

        # Update best
        if sil_score > results['best_silhouette']:
            results['best_k'] = k
            results['best_silhouette'] = sil_score

    print(f"\n✅ Optimal k = {results['best_k']} with silhouette score = {results['best_silhouette']:.4f}")

    return results, scaler



In [26]:
def clean_full_dataset_toxic_combinations(data, toxic_combinations):
    """
    Efficiently clean the full dataset by removing toxic entitlements.

    Args:
        data: Original DataFrame
        toxic_combinations: Dict from load_toxic_combinations_from_single_file

    Returns:
        cleaned_data: DataFrame with toxic entitlements removed
        removal_stats: Statistics about removals
    """
    print("🧹 Cleaning full dataset by removing toxic entitlements...")

    cleaned_data = data.copy()
    removal_stats = {
        'total_removals': 0,
        'employees_affected': set(),
        'departments_affected': {},
        'entitlements_removed': set()
    }

    # Process each department's toxic combinations
    for dept, dept_toxic in toxic_combinations.items():
        if 'name_department' not in cleaned_data.columns:
            print(f"⚠️  No department column found, treating all as one department")
            dept_mask = np.ones(len(cleaned_data), dtype=bool)
        else:
            dept_mask = cleaned_data['name_department'] == dept

        dept_indices = np.where(dept_mask)[0]

        if len(dept_indices) == 0:
            continue

        print(f"  Processing {dept}: {len(dept_toxic)} toxic pairs")
        dept_removals = 0

        # Vectorized processing for efficiency
        for (ent1, ent2), entitlement_to_remove in dept_toxic.items():
            if all(col in cleaned_data.columns for col in [ent1, ent2, entitlement_to_remove]):
                # Find employees with both entitlements using numpy
                both_mask = (
                    (cleaned_data[ent1].values == 1) &
                    (cleaned_data[ent2].values == 1) &
                    dept_mask
                )

                affected_indices = np.where(both_mask)[0]

                if len(affected_indices) > 0:
                    # Vectorized removal
                    cleaned_data.iloc[affected_indices, cleaned_data.columns.get_loc(entitlement_to_remove)] = 0

                    dept_removals += len(affected_indices)
                    removal_stats['employees_affected'].update(
                        cleaned_data.iloc[affected_indices]['employee_id'].values
                    )
                    removal_stats['entitlements_removed'].add(entitlement_to_remove)

        removal_stats['departments_affected'][dept] = dept_removals
        removal_stats['total_removals'] += dept_removals

    # Update total_entitlements column if exists
    if 'total_entitlements' in cleaned_data.columns:
        entitlement_cols = [col for col in cleaned_data.columns if col.startswith('ENT') and col[3:].isdigit()]
        cleaned_data['total_entitlements'] = cleaned_data[entitlement_cols].sum(axis=1)

    print(f"\n✅ Cleaning complete:")
    print(f"   Total removals: {removal_stats['total_removals']}")
    print(f"   Employees affected: {len(removal_stats['employees_affected'])}")
    print(f"   Unique entitlements removed: {len(removal_stats['entitlements_removed'])}")

    return cleaned_data, removal_stats



In [27]:
def create_3d_comparison_plot(X_original, X_cleaned, labels_original, labels_cleaned,
                            metadata, results_original, results_cleaned, removal_stats):
    """
    Create an efficient side-by-side 3D PCA comparison plot.
    """
    print("🎨 Creating 3D comparison visualization...")

    # Create subplots
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=(
            f"Original Data (k={results_original['best_k']}, Silhouette={results_original['best_silhouette']:.3f})",
            f"Cleaned Data (k={results_cleaned['best_k']}, Silhouette={results_cleaned['best_silhouette']:.3f})"
        ),
        specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}]],
        horizontal_spacing=0.05
    )

    # Colors
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8',
              '#F7DC6F', '#BB8FCE', '#85C1E2', '#F8B739', '#82E0AA']

    # Process both datasets
    datasets = [
        (X_original, labels_original, "Original", 1),
        (X_cleaned, labels_cleaned, "Cleaned", 2)
    ]

    for X, labels, name, col in datasets:
        # PCA with optimized components
        n_components = min(3, X.shape[0] - 1, X.shape[1])
        pca = PCA(n_components=n_components, svd_solver='randomized')  # Faster for large datasets

        # Scale and transform
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        X_pca = pca.fit_transform(X_scaled)

        # Pad if needed
        if n_components < 3:
            X_pca = np.pad(X_pca, ((0, 0), (0, 3 - n_components)), mode='constant')

        # Add traces for each cluster
        unique_labels = np.unique(labels)

        for cluster_id in unique_labels:
            mask = labels == cluster_id

            # Downsample for visualization if too many points
            indices = np.where(mask)[0]
            if len(indices) > 1000:
                indices = np.random.choice(indices, 1000, replace=False)

            # Create hover text for sample
            hover_texts = []
            for idx in indices[:100]:  # Limit hover text for performance
                dept = metadata.iloc[idx].get('name_department', 'Unknown')
                risk = metadata.iloc[idx].get('avg_risk', 0)
                hover_texts.append(f"Cluster {cluster_id}<br>Dept: {dept}<br>Risk: {risk:.3f}")

            fig.add_trace(
                go.Scatter3d(
                    x=X_pca[indices, 0],
                    y=X_pca[indices, 1],
                    z=X_pca[indices, 2],
                    mode='markers',
                    marker=dict(
                        color=colors[cluster_id % len(colors)],
                        size=4,
                        opacity=0.6
                    ),
                    name=f'{name} C{cluster_id} ({mask.sum()})',
                    text=hover_texts[:len(indices)],
                    hovertemplate='%{text}<extra></extra>',
                    showlegend=(col == 1)
                ),
                row=1, col=col
            )

    # Update layout
    fig.update_layout(
        title={
            'text': f'Full Employee Clustering: {removal_stats["total_removals"]} Toxic Entitlements Removed<br>' +
                   f'<sup>Silhouette Improvement: {results_cleaned["best_silhouette"] - results_original["best_silhouette"]:+.3f}</sup>',
            'x': 0.5,
            'xanchor': 'center'
        },
        template='plotly_dark',
        height=700,
        width=1400,
        showlegend=True,
        legend=dict(x=1.02, y=0.5)
    )

    # Update 3D scene layouts
    for i in range(1, 3):
        fig.update_scenes(
            dict(
                xaxis=dict(title='PC1', gridcolor='gray', showbackground=False),
                yaxis=dict(title='PC2', gridcolor='gray', showbackground=False),
                zaxis=dict(title='PC3', gridcolor='gray', showbackground=False),
                camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
            ),
            row=1, col=i
        )

    return fig



In [28]:
def run_efficient_full_clustering_analysis(data, toxic_combinations_path=None, k_range=None):
    """
    Efficient full employee clustering analysis with before/after toxic removal comparison.

    Args:
        data: Full employee DataFrame (already binarized with ENT columns)
        toxic_combinations_path: Path to toxic combinations file
        k_range: Optional list of k values to test

    Returns:
        Dict with all results and visualizations
    """
    print("🚀 Starting efficient full employee clustering analysis")
    print("=" * 70)

    # Load toxic combinations if path provided
    if toxic_combinations_path:
        from clustering_pre_post_toxicdetection import load_toxic_combinations_from_single_file
        toxic_combinations = load_toxic_combinations_from_single_file(toxic_combinations_path)
    else:
        print("⚠️  No toxic combinations path provided, using empty dict")
        toxic_combinations = {}

    # Step 1: Prepare original data
    print("\n1️⃣ Preparing original data")
    X_original, feature_cols, metadata = prepare_full_employee_data(data)

    # Step 2: Find optimal k for original
    print("\n2️⃣ Finding optimal k for original data")
    results_original, scaler_original = find_optimal_k_efficient(X_original, k_range)

    # Step 3: Clean data
    print("\n3️⃣ Cleaning data")
    if toxic_combinations:
        cleaned_data, removal_stats = clean_full_dataset_toxic_combinations(data, toxic_combinations)
    else:
        cleaned_data = data.copy()
        removal_stats = {'total_removals': 0, 'employees_affected': set()}

    # Step 4: Prepare cleaned data
    print("\n4️⃣ Preparing cleaned data")
    X_cleaned, _, _ = prepare_full_employee_data(cleaned_data)

    # Step 5: Find optimal k for cleaned
    print("\n5️⃣ Finding optimal k for cleaned data")
    results_cleaned, scaler_cleaned = find_optimal_k_efficient(X_cleaned, k_range)

    # Get optimal labels
    labels_original = results_original['cluster_labels'][results_original['best_k']]
    labels_cleaned = results_cleaned['cluster_labels'][results_cleaned['best_k']]

    # Step 6: Create visualization
    print("\n6️⃣ Creating comparison visualization")
    fig_comparison = create_3d_comparison_plot(
        X_original, X_cleaned, labels_original, labels_cleaned,
        metadata, results_original, results_cleaned, removal_stats
    )

    # Summary
    improvement = results_cleaned['best_silhouette'] - results_original['best_silhouette']

    print("\n" + "=" * 70)
    print("📊 ANALYSIS SUMMARY:")
    print(f"Original Data:")
    print(f"  - Optimal k: {results_original['best_k']}")
    print(f"  - Silhouette: {results_original['best_silhouette']:.4f}")
    print(f"Cleaned Data:")
    print(f"  - Optimal k: {results_cleaned['best_k']}")
    print(f"  - Silhouette: {results_cleaned['best_silhouette']:.4f}")
    print(f"Improvement: {improvement:+.4f} ({improvement/results_original['best_silhouette']*100:+.1f}%)")
    print("=" * 70)

    return {
        'original': {
            'X': X_original,
            'results': results_original,
            'labels': labels_original,
            'scaler': scaler_original
        },
        'cleaned': {
            'X': X_cleaned,
            'results': results_cleaned,
            'labels': labels_cleaned,
            'scaler': scaler_cleaned,
            'data': cleaned_data
        },
        'comparison': {
            'figure': fig_comparison,
            'improvement': improvement,
            'removal_stats': removal_stats
        },
        'metadata': metadata,
        'feature_cols': feature_cols
    }



In [29]:
def analyze_full_employee_clustering(data, toxic_combinations_path="department_anomaly_results/all_departments_combined.csv"):
    """
    Quick function to run the analysis with your data structure.

    Usage:
        results = analyze_full_employee_clustering(data)
        results['comparison']['figure'].show()
    """
    # If data is not binarized, you'll need to binarize it first
    if 'ENT001' not in data.columns:
        print("⚠️  Data doesn't appear to be binarized. Please binarize first.")
        return None

    return run_efficient_full_clustering_analysis(data, toxic_combinations_path)


In [30]:
results = analyze_full_employee_clustering(binarized_df)

🚀 Starting efficient full employee clustering analysis
🔍 Loading toxic combinations from single file: 'department_anomaly_results/all_departments_combined.csv'...
📊 Loaded 204814 combinations from all_departments_combined.csv
📋 Columns: ['combination_id', 'entitlement_1', 'entitlement_2', 'entitlement_1_name', 'entitlement_2_name', 'frequency', 'log_frequency', 'is_rare', 'max_risk', 'min_risk', 'avg_risk', 'risk_difference', 'both_high_risk', 'max_criticality', 'avg_criticality', 'same_application', 'same_type', 'same_function', 'cross_function_high_risk', 'combined_risk_score', 'cluster', 'distance_to_center', 'is_anomaly', 'anomaly_score', 'batch_id', 'processing_timestamp', 'department', 'department_employees', 'department_entitlements', 'toxic']
✅ Using 'department' as department column
  🔄 Processing Accounting...
    🚨 Found 122 TOXIC combinations (anomalous + same app)
    ✅ Accounting: 122 toxic entitlement pairs to clean
  🔄 Processing Application Development...
    🚨 Found 1

In [31]:
results['comparison']['figure'].show()